In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("abalone.csv")
df.head()

,Type,LongestShell,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [ ]:
data=pd.DataFrame()
data["x"]=df["LongestShell"]
data["y"]=df["Diameter"]
data.head()

,x,y
0,0.455,0.365
1,0.350,0.265
2,0.530,0.420
3,0.440,0.365
4,0.330,0.255


In [ ]:
!pip install Pyfhel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for Pyfhel: filename=Pyfhel-3.4.2-cp310-cp310-linux_x86_64.whl size=8501513 sha256=4e458aedecf3b9dd0b52a31841a4c93772419ed46086a9339eda2ba05d1d7279
  Stored in directory: /root/.cache/pip/wheels/74/7b/34/876b6d79c02f65e68e4e6f262735c2347cf77581ac50ca470d
Successfully built Pyfhel


In [ ]:
import math
import numpy as np
from Pyfhel import Pyfhel, PyCtxt
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def distance(d1,d2):
    return math.sqrt(math.pow(float(d1),2)+ math.pow(float(d2),2))

def l2_dist_sq_ctxt(vec_1: PyCtxt, vec_2: PyCtxt, HE: Pyfhel):
    """
    Calculates the encrypted squared l2 norm between encrypted vectors vec_1 and vec_2
    """
    dist_sq = ~(vec_1 - vec_2)**2
    HE.relinKeyGen()  # relinearize after ctxt-ctxt mults
    #dist_sq = HE.cumul_add(dist_sq)
    return dist_sq


m = 100
P = [Point(0, 0) for _ in range(m)]

for i in range(m):
    P[i].x, P[i].y = data.loc[i, 'x'], data.loc[i, 'y']

k = int(input("Enter the number of clusters: "))

px=[]
py=[]
for i in range(m):
    px.append(P[i].x)
    py.append(P[i].y)

HE = Pyfhel()
ckks_params = {'scheme': 'CKKS', 'n': 8192, 'scale': 2**40, 'qi_sizes': [60, 40, 40, 60]}
HE.contextGen(**ckks_params)
HE.keyGen()
HE.rotateKeyGen()

crp = [Point(0,0) for _ in range(k)]
cp = [Point(0,0) for i in range(m)]

for j in range(0,m):
    cp[j].x=HE.encrypt(px[j:j+HE.get_nSlots()])
    cp[j].y=HE.encrypt(py[j:j+HE.get_nSlots()])
rtemp=[]
pcost=float('inf')
res=[]
x=100
fcost=0
while x>0:
  print(x)
  rp_indices = np.random.choice(m, size=k, replace=False).tolist()  # Randomly select k indices from the dataset
  while rp_indices in rtemp:
    rp_indices = np.random.choice(m, size=k, replace=False).tolist()
  rtemp.append(rp_indices)
  rp = [Point(0, 0) for _ in range(k)]
  for i in range(k):
    rp[i].x, rp[i].y = data.loc[rp_indices[i], 'x'], data.loc[rp_indices[i], 'y']
  rx=[]
  ry=[]
  for i in range(k):
    rx.append(rp[i].x)
    ry.append(rp[i].y)
  for j in range(0,k):
    crp[j].x=HE.encrypt(rx[j:j+HE.get_nSlots()])
    crp[j].y=HE.encrypt(ry[j:j+HE.get_nSlots()])
  arr = [[] for _ in range(k)]
  mp = Point(0,0)
  count = [0] * k
  cost=0
  for i in range(m):
    d=[]
    for j in range(k):
      d1 = HE.decrypt(l2_dist_sq_ctxt(cp[i].x, crp[j].x, HE))
      d2 = HE.decrypt(l2_dist_sq_ctxt(cp[i].y, crp[j].y, HE))
      d.append(distance(d1[0],d2[0]))
    c = d.index(min(d))
    cost+=min(d)
    arr[c].append(P[i])
    count[c] += 1

  if cost==pcost:
    print("Success")
    print(res)
    break
  elif cost>pcost:
    print("Cost greater...Leave this and go to next")
  else:
    print("Cost lesser...")
    pcost=cost
    fcost=cost
    res=arr
  x-=1
s=1
print("Cost: ",fcost)
for i in res:
  print("Cluster: ",s)
  for j in i:
    print(j.x," ",j.y)
  s+=1




100


<ipython-input-10-8dc2abd4b093>:16: RuntimeWarning: <Pyfhel Warning> relin_key empty, generating it for relinearization.
  dist_sq = ~(vec_1 - vec_2)**2


Cost lesser...
99
Cost lesser...
98
Cost lesser...
97
Cost greater...Leave this and go to next
96
Cost lesser...
95
Cost greater...Leave this and go to next
94
Cost greater...Leave this and go to next
93
Cost greater...Leave this and go to next
92
Cost lesser...
91
Cost lesser...
90
Cost greater...Leave this and go to next
89
Cost greater...Leave this and go to next
88
Cost greater...Leave this and go to next
87
Cost greater...Leave this and go to next
86
Cost greater...Leave this and go to next
85
Cost greater...Leave this and go to next
84
Cost greater...Leave this and go to next
83
Cost greater...Leave this and go to next
82
Cost greater...Leave this and go to next
81
Cost greater...Leave this and go to next
80
Cost greater...Leave this and go to next
79
Cost greater...Leave this and go to next
78
Cost greater...Leave this and go to next
77
Cost greater...Leave this and go to next
76
Cost greater...Leave this and go to next
75
Cost greater...Leave this and go to next
74
Cost greater